# Normalization

Normalization forces all inputs to play by the same rules. We usually want them to have:

- **Mean (average) = 0** (Centered around zero).
- **Standard Deviation (spread) = 1** (Most values between -1 and 1).

## The Math (The Z-Score Formula)
To normalize a value $x$:

$$
x_{\text{norm}} = \frac{x - \text{mean}}{\text{standard\_deviation}}
$$

- **Subtract the mean:** Moves the center of the data to 0.
- **Divide by std dev:** Squishes (or stretches) the data so the spread is 1.

Now, "Number of Rooms" and "Square Footage" both look like numbers between -1 and 1. The network treats them equally.